In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import *
from tensorflow.keras import Model, Input, regularizers
from tensorflow.keras.models import Sequential
import os
from tensorflow.keras.callbacks import EarlyStopping
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=50, verbose=1, mode='min')
from tensorflow.keras.models import load_model
BACKEND = "tensorflow"
os.environ['KERAS_BACKEND'] = BACKEND
os.environ['THEANO_FLAGS'] = "device=gpu0, lib.cnmem=0.85, optimizer=fast_run"
import warnings
warnings.filterwarnings("ignore")

import import_ipynb
import prepare_data as pd

c:\users\zyana\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\zyana\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\zyana\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\zyana\appd

importing Jupyter notebook from prepare_data.ipynb


In [3]:
X_train, X_test, y_train, y_test = pd.get_data()
X_train = X_train.astype('float32') / 255.
X_test = X_test.astype('float32') / 255.
y_train = y_train.astype('float32') / 255.
y_test = y_test.astype('float32') / 255.

In [2]:
w = 852
h = 480
c = 3

In [3]:
Input_img = Input(shape=(h, w, c))  
    
x1 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(Input_img)
x2 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x1)
x3 = MaxPool2D(padding='same')(x2)
x4 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x3)
x5 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x4)
x6 = MaxPool2D(padding='same')(x5)
encoded = Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x6)

x7 = UpSampling2D()(encoded)
x8 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x7)
x9 = Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x8)
x10 = Add()([x5, x9])
x11 = UpSampling2D()(x10)
x12 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x11)
x13 = Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(10e-10))(x12)
x14 = Add()([x2, x13])

decoded = Conv2D(3, (3, 3), padding='same',activation='relu', kernel_regularizer=regularizers.l1(10e-10))(x14)
autoencoder = Model(Input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [4]:
autoencoder.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 480, 852, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 480, 852, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 480, 852, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 240, 426, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
history = autoencoder.fit(X_train[0:300], y_train[0:300],
            epochs=5,
            batch_size=3,
            validation_data=(X_test[0:100], y_test[0:100]),
            callbacks=[early_stopper])
autoencoder.save('autoencoder.h5')
autoencoder.save_weights('autoencoder_weights.h5')

Train on 300 samples, validate on 100 samples
Epoch 1/5
300/300 [==============================] - 4703s 16s/sample - loss: 0.0237 - accuracy: 0.6530 - val_loss: 7.3045e-04 - val_accuracy: 0.9111
Epoch 2/5
300/300 [==============================] - 4722s 16s/sample - loss: 4.6483e-04 - accuracy: 0.9400 - val_loss: 3.0213e-04 - val_accuracy: 0.9631
Epoch 3/5
300/300 [==============================] - 4795s 16s/sample - loss: 3.1287e-04 - accuracy: 0.9608 - val_loss: 2.7978e-04 - val_accuracy: 0.9583
Epoch 4/5
300/300 [==============================] - 4770s 16s/sample - loss: 2.8384e-04 - accuracy: 0.9654 - val_loss: 2.7733e-04 - val_accuracy: 0.9659
Epoch 5/5
300/300 [==============================] - 4713s 16s/sample - loss: 2.7046e-04 - accuracy: 0.9660 - val_loss: 2.4890e-04 - val_accuracy: 0.9697


In [2]:
autoencoder = load_model('autoencoder.h5')
autoencoder.load_weights('autoencoder_weights.h5')

In [40]:
x = pd.read_images('in/frames240')
x = x.astype('float32') / 255.
pd.save_images(autoencoder.predict(x), 'autoencoder')

In [2]:
pd.frames_to_video('out/autoencoder', 'autoencoder')